Outcome variables:

- depression
- anxiety
- parenting stress

Confounders

- mental health treatment
- Child level variables
    - expressive language
    - ADOS
- DAS (relationships)

Start with time 1 versus time 3
- finished at least 10 sessions (`finish_tx`)

Similar age and location of kids

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt

In [3]:
mbsr_raw = pd.read_csv('../data/MBSR Data Set 3-28-18 for analysis.csv', index_col=0, na_values=[' '],
                       parse_dates=['child_dob', 'ados_date'])
mbsr_raw.tail()

,finish_tx,parentfinish_t6,childfinish_t6,child_dob,ados_date,age_ados,ados_module,ados_new_algorithm,ados_severity,ados_concern,...,DescribeT5,AwareT5,NonjudgeT5,NonreactT5,FFMQT6,ObserveT6,DescribeT6,AwareT6,NonjudgeT6,NonreactT6
study_id,,,,,,,,,,,,,,,,,,,,,
59,1,1.0,1,2015-05-07,2017-04-04,1.91,3.0,17.0,NaN,3.0,...,NaN,NaN,NaN,NaN,126.0,16.0,33.0,28.0,29.0,20.0
60,1,0.0,1,2015-02-17,2017-04-21,2.17,3.0,26.0,NaN,3.0,...,26.0,33.0,28.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
61,0,0.0,0,2014-10-20,2017-04-22,2.51,3.0,17.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,1,1.0,1,2014-12-06,2017-05-22,2.46,3.0,18.0,NaN,3.0,...,25.0,32.0,32.0,14.0,117.0,19.0,21.0,28.0,33.0,16.0
63,1,1.0,1,2015-03-19,2017-06-26,2.27,3.0,22.0,NaN,3.0,...,27.0,32.0,33.0,17.0,131.0,29.0,24.0,31.0,32.0,15.0


In [5]:
columns_to_keep = ['finish_tx', 'age_ados']
mbsr_data = mbsr_raw[columns_to_keep]
mbsr_data.head()

,finish_tx
study_id,
1,1
2,1
3,1
4,1
5,1
